In [4]:
import pandas as pd
import defusedxml.ElementTree as ET
import re

In [5]:
tree = ET.parse('vrejcelk.xml')

In [6]:
root = tree.getroot()

In [7]:
data = []
# list pro budoucí část kodu
# obor_seznam = []

for ps in root.findall('.//PravniSubjekt'):
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text

    """ někde lze v datech jít ještě o úroveň níže a vytáhnout podrobnosti např. o 'SkolaOboryVzdelani'
    pro naše účely údaje (prozatím) nepotřebujeme, v případě jejich použití by nám v datech vznikly
    duplicity!! V 'zazizeniKapacita' figuruje součet kapacit ze skupin 'SkolaOboryVzdelani'"""
    
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            # v datech nejsou adresy vyplněné dle stejného vzoru, někde jsou none hodnoty, někde jen 
            # název obce, někde psc + název obce, proto je nutné zaznamenat obě dvě varianty, respektive sloupce
            idMista = smvc.find('IDMista').text
            adresaX = (smvc.find('MistoAdresa2').text)
            adresaY = smvc.find('MistoAdresa3').text
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace
            """ adresu možná nahradit přes RUIAN API, nebo https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2F00025712%2F59ddccd404a617f695e3a1ef8e65b81e 
            záleží kam se bude kod a model vyvíjet """

            """ NUTNO OVĚŘIT-POŘEŠIT -> JAK SE SCRAPUJÍ MÍSTA VÝKONU ČINNOSTI, ABY SE NEJEDNALO O DUPLICITY A 
            MĚLI JSME KOMPLETNÍ SEZNAM MÍST """ 

        
            """ Předpřipravená část kodu pro scrap dat k roztřízení škol na základní a střední vzdělávání (konzervatoře, gymnázia)
            nutno se poté vypořádat s případnými duplicitami, či data uložit do samostané tabulky a pracovat s nimi zvlášť """
        
            # for sov in smvc.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
            #     oborKod = sov.find('OborKod')
            #     oborNazev = sov.find('OborNazev')
            #     delkaVzdelavani = sov.find('DelkaVzdelavani')

        zarizeni = {
                    'IZO' : izo, 
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniTyp': typZarizeni, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka, 
                    'jazyk' : jazyk, 
                    'IDMista': idMista,
                    'adresaX' : adresaX,
                    'adresaY' : adresaY,
                    'druh' : druh
                   }
        data.append(zarizeni)

        # budoucí část kodu
        # delkavzdelanani = {
        #             'IZO' : izo, 
        #             'OborKod': oborKod,
        #             'OborNazev': oborNazev,
        #             'DelkaVzdelavaní': delkaVzdelavani,
        # }
        # obor_seznam.append(delkavzdelanani)

In [8]:
data[:5]

[{'IZO': '049625918',
  'ICO': '49625918',
  'ReditelstviNazev': 'Mateřská škola sv. Voršily v Praze',
  'ZarizeniTyp': 'Mateřská škola',
  'ZarizeniDruhTyp': 'A00',
  'Kapacita': '54',
  'KapacitaJednotka': 'dítě',
  'jazyk': 'český',
  'IDMista': '049625918',
  'adresaX': 'Nové Město',
  'adresaY': '110 00 Praha 1',
  'druh': 'Mateřská škola'},
 {'IZO': '102413096',
  'ICO': '49625918',
  'ReditelstviNazev': 'Mateřská škola sv. Voršily v Praze',
  'ZarizeniTyp': 'Školní jídelna',
  'ZarizeniDruhTyp': 'L11',
  'Kapacita': '90',
  'KapacitaJednotka': 'stravovaný',
  'jazyk': 'český',
  'IDMista': '102413096',
  'adresaX': 'Nové Město',
  'adresaY': '110 00 Praha 1',
  'druh': 'Školní jídelna'},
 {'IZO': '107500884',
  'ICO': '61379310',
  'ReditelstviNazev': 'Církevní mateřská škola Studánka',
  'ZarizeniTyp': 'Mateřská škola',
  'ZarizeniDruhTyp': 'A00',
  'Kapacita': '70',
  'KapacitaJednotka': 'dítě',
  'jazyk': 'český',
  'IDMista': '107500884',
  'adresaX': 'Kamýk',
  'adresaY': '

In [9]:
df = pd.DataFrame(data)

In [10]:
df.head()
# df.shape #27190
# df['IZO'].value_counts() #27190, oveření, zda je 'IZO' PK - unikátní identifikátor

,IZO,ICO,ReditelstviNazev,ZarizeniTyp,ZarizeniDruhTyp,Kapacita,KapacitaJednotka,jazyk,IDMista,adresaX,adresaY,druh
0,049625918,49625918,Mateřská škola sv. Voršily v Praze,Mateřská škola,A00,54,dítě,český,049625918,Nové Město,110 00 Praha 1,Mateřská škola
1,102413096,49625918,Mateřská škola sv. Voršily v Praze,Školní jídelna,L11,90,stravovaný,český,102413096,Nové Město,110 00 Praha 1,Školní jídelna
2,107500884,61379310,Církevní mateřská škola Studánka,Mateřská škola,A00,70,dítě,český,107500884,Kamýk,142 00 Praha 4,Mateřská škola
3,161102263,61379310,Církevní mateřská škola Studánka,Školní jídelna - výdejna,L13,70,stravovaný,český,161102263,Kamýk,142 00 Praha 4,Školní jídelna - výdejna
4,110034384,25765710,Modrý klíč - základní škola speciální a mateřs...,Mateřská škola,A00,6,dítě,český,110034384,Kamýk,142 00 Praha 4,Mateřská škola pro děti se zdravotním postižením


In [11]:
df.info() # nulové hodnoty jsou především u adres, kde pro jistotu scrapujeme 2 informace o odrese
# kapacita zařízení figuruje všude

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27190 entries, 0 to 27189
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               27190 non-null  object
 1   ICO               27190 non-null  object
 2   ReditelstviNazev  27190 non-null  object
 3   ZarizeniTyp       27190 non-null  object
 4   ZarizeniDruhTyp   27190 non-null  object
 5   Kapacita          27190 non-null  object
 6   KapacitaJednotka  27130 non-null  object
 7   jazyk             27190 non-null  object
 8   IDMista           27190 non-null  object
 9   adresaX           27190 non-null  object
 10  adresaY           8547 non-null   object
 11  druh              27188 non-null  object
dtypes: object(12)
memory usage: 2.5+ MB


In [12]:
# kapacitanull = df[df['KapacitaJednotka'].isna()]
# kapacitanull['ZarizeniTyp'].value_counts() # chybějící data o KapacitaJednotka figurují i např. v Mateřské škole, Základní škole i Střední škole
# kapacitanull[kapacitanull['ZarizeniTyp'].isin(['Mateřská škola', 'Základní škola', 'Střední škola'])]

In [13]:
df['ZarizeniTyp'].value_counts()
# df['ZarizeniTyp'].value_counts().count() # počet unikátních hodnot
# df['ZarizeniTyp'].unique() # seznam v listu
# df['jazyk'].value_counts()

ZarizeniTyp
Školní jídelna                                            5938
Mateřská škola                                            5365
Základní škola                                            4353
Školní družina                                            4229
Školní jídelna - výdejna                                  2914
Střední škola                                             1332
Školní klub                                                730
Základní umělecká škola                                    516
Domov mládeže                                              383
Dům dětí a mládeže                                         267
Vyšší odborná škola                                        166
Dětský domov                                               137
Speciálně pedagogické centrum                              107
Mateřská škola (lesní mateřská škola)                      104
Internát                                                    70
Zařízení pro další vzdělávání pedagogických

In [14]:
# df[df['ZarizeniTyp'] == 'Střední škola']['druh'].value_counts() #členění středních škol 

In [15]:
# df[(df['ZarizeniTyp'] == 'Střední škola') & (df['druh'].isin(['Gymnázium', 'Konzervatoř']))] 
# blbost, konzervatoř totiž není v ZarizeniTyp ve střední škole, ale jako konzervatoř

df[(df['ZarizeniTyp'] == 'Přípravný stupeň základní školy speciální')]

,IZO,ICO,ReditelstviNazev,ZarizeniTyp,ZarizeniDruhTyp,Kapacita,KapacitaJednotka,jazyk,IDMista,adresaX,adresaY,druh
2917,110400887,60254190,"Základní škola, Jablonec nad Nisou, Liberecká ...",Přípravný stupeň základní školy speciální,M60,6,žák,český,167100432,Mšeno nad Nisou,466 04 Jablonec nad Nisou,Přípravný stupeň základní školy speciální
3283,110500652,61716464,"Dětský domov, Mateřská škola, Základní škola a...",Přípravný stupeň základní školy speciální,M60,12,žák,český,110500652,760 01 Zlín,None,Přípravný stupeň základní školy speciální
3290,150068816,61716413,"Základní škola Otrokovice, Komenského",Přípravný stupeň základní školy speciální,M60,6,žák,český,150068816,765 02 Otrokovice,None,Přípravný stupeň základní školy speciální
3293,110500971,61716472,"Základní škola Slavičín, Hrádek na Vlárské dráze",Přípravný stupeň základní školy speciální,M60,6,žák,český,110500971,Hrádek na Vlárské dráze,763 21 Slavičín,Přípravný stupeň základní školy speciální
3327,110500458,47934581,"Základní škola Kroměříž, 1. máje",Přípravný stupeň základní školy speciální,M60,20,žák,český,110500458,767 01 Kroměříž,None,Přípravný stupeň základní školy speciální
3341,110500415,47935928,"Základní škola a Mateřská škola Kroměříž, F. V...",Přípravný stupeň základní školy speciální,M60,12,žák,český,110500415,767 01 Kroměříž,None,Přípravný stupeň základní školy speciální
3369,150044321,60371668,Základní škola a Mateřská škola Uherské Hradiš...,Přípravný stupeň základní školy speciální,M60,6,žák,český,181004801,686 06 Uherské Hradiště,None,Přípravný stupeň základní školy speciální
3380,110037227,60370432,Základní škola a Mateřská škola Uherské Hradiš...,Přípravný stupeň základní školy speciální,M60,18,žák,český,173100741,687 41 Medlovice,None,Přípravný stupeň základní školy speciální
3623,110501136,70238910,"Základní škola Rožnov pod Radhoštěm, Tyršovo n...",Přípravný stupeň základní školy speciální,M60,10,žák,český,110501136,756 61 Rožnov pod Radhoštěm,None,Přípravný stupeň základní školy speciální
21261,110501101,70238898,"Základní škola, Mateřská škola a Praktická ško...",Přípravný stupeň základní školy speciální,M60,9,žák,český,110501101,755 01 Vsetín,None,Přípravný stupeň základní školy speciální


## MŠ + PSZŠSpeciální TAB

In [15]:
# materske_skoly = df[df["ZarizeniTyp"].isin(["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v."])]
# materske_skoly.loc[:, 'Kapacita'] = materske_skoly['Kapacita'].astype(int)
# materske_skoly['Kapacita'].sum()

materska_skola_SUM = df.loc[df["ZarizeniTyp"] == "Mateřská škola", 'Kapacita'].astype(int).sum()
materska_skola_LES_SUM = df.loc[df["ZarizeniTyp"] == "Mateřská škola (lesní mateřská škola)", 'Kapacita'].astype(int).sum()
materska_skola_POL_SUM = df.loc[df["ZarizeniTyp"].isin(["Mateřská škola - Przedszkole", "Mateřská škola s p.j.v."]), 'Kapacita'].astype(int).sum()
pripravny_stupenZS_SUM = df.loc[df["ZarizeniTyp"] == "Přípravný stupeň základní školy speciální", "Kapacita"].astype(int).sum()
""" Přípravný stupeň umožňuje vzdělávání dítěte od školního roku, v němž dosáhne 5 let věku
nejdéle však do 8 let věku. Tzn. že v přípravném stupni jsou vzdělávány děti před 
započetím povinné školní docházky, jde tudíž o vzdělávání předškolní. - https://skola-ostrava.diakonie.cz/nase-skola/soucasti-skoly/pripravny-stupen/ """
pass

In [16]:
materske_skoly = df[df["ZarizeniTyp"].isin(["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"])]
materske_skoly.loc[:, 'Kapacita'] = materske_skoly['Kapacita'].astype(int)
# materske_skoly['Kapacita'].sum
# materske_skoly
# kapacitaMS = materska_skola_SUM + materska_skola_LES_SUM + materska_skola_POL_SUM + pripravny_stupenZS_SUM
# kapacitaMS

## DětskéSkupinky TAB

In [63]:
from web_scrap_DS import DS_tabulka

In [70]:
DS_tabulka

,Region,Kapacita
0,Hlavní město Praha,4361
1,Jihočeský,1305
2,Jihomoravský,3981
3,Karlovarský,256
4,Královéhradecký,1113
5,Liberecký,734
6,Moravskoslezský,2186
7,Olomoucký,1339
8,Pardubický,1044
9,Plzeňský,1037


## Porodnost 2018-2023 TAB 

In [15]:
from web_scrap_porodnost import porodnost_tabulka

In [62]:
porodnost_tab = porodnost_tabulka.astype({"PocetNarozenychDeti":'int'}) 
porodnost_tabCR = porodnost_tab[porodnost_tab['Kraj'] == 'Česká republika']
porodnost_tab_kraje = porodnost_tab[porodnost_tab['Kraj'] != 'Česká republika']
# porodnost_tab_kraje
# porodnost_tabCR[porodnost_tabCR['Rok'].isin([2021, 2020, 2019, 2018])]['PocetNarozenychDeti'].sum()